In [1]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import transformers
import torch
import json
from peft import PeftModel

In [2]:
dataset = "games"
base_model = "../../models/LLMs/LLaMa-7B"
device = "cuda:0"

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(base_model)
model = LlamaForCausalLM.from_pretrained(
            base_model,
            load_in_8bit=False,
            torch_dtype=torch.float16,
    ).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2
model.eval()

if dataset == "games":
    f = open(f'../../datasets/{dataset}/SeqRec/id2name.txt', 'r')
    lines = f.readlines()
    f.close()
    text = [_.split('\t')[0].strip(" ").strip('\"') for _ in lines] # remove the leading and trailing spaces and quotess make sure this preprocess is the same as the prediction
    tokenizer.padding_side = "left"
else:
    f = open(f'../../datasets/{dataset}/SeqRec/id2name.json', 'r')
    text = list(json.load(f).values())
    tokenizer.padding_side = "left"

In [ ]:
from tqdm import tqdm
def batch(list, batch_size=1):
    chunk_size = (len(list) - 1) // batch_size + 1
    for i in range(chunk_size):
        yield list[batch_size * i: batch_size * (i + 1)]
item_embedding = []
with torch.no_grad():
    for i, batch_input in tqdm(enumerate(batch(text, 16))):
        input = tokenizer(batch_input, return_tensors="pt", padding=True)
        input_ids = input.input_ids.cuda()
        attention_mask = input.attention_mask.cuda()
        outputs = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states
        item_embedding.append(hidden_states[-1][:, -1, :].detach().cpu())
    # break
    item_embedding = torch.cat(item_embedding, dim=0)
    torch.save(item_embedding, f'../../datasets/{dataset}/SeqRec/item_embedding.pt')

204it [00:15, 12.52it/s]

In [ ]:
item_embedding.shape